In [2]:
! pip install pathling

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=c9a5e2911b75829da3fb27c8c69930c1f9ed6ab2083664aa70100fb7500c2131
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [3]:
! pip install psycopg2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# valueBoolean changes
# for the second option trial of creating a table and config file via code,
# inserting as much as possible from fhir ,then through pathling code

import requests
import json
import psycopg2
from pathling import PathlingContext
from tempfile import NamedTemporaryFile
import os


resourceArray=["Observation"]


conn = psycopg2.connect(database="realtime_export_fhirtest2", user = "eform_user", password = "eform@123$", host = "3.22.85.62",port = "5432")
print("Opened database successfully")
print("length of resource array",len(resourceArray))
cur = conn.cursor()

for resource in resourceArray:
    resourceName = resource
    
    url = "https://fhirtest2.intercorpvt.com:8000/fhir/" + resourceName
    headers = {'Accept': 'application/fhir+json'}
    response = requests.get(url, headers=headers)
    # Save the response to a temporary file.
    # f = open("file", "w")
    f = open("file", "w", encoding="utf-8")
    f.write(response.text)
    f.close()

    # Read the Bundle into Spark.
    pc = PathlingContext.create()
    bundle = pc.spark.read.text("file", wholetext=True)

    
    EncodedBundle = pc.encode_bundle(bundle, resourceName)

    pt = EncodedBundle.schema
    data1 = EncodedBundle.schema.json()

    jsonData = data1.replace("true", "True")
    jsondata = json.loads(data1)
    fhirResourceDescp = "Resource type is"
    resNameDesc = fhirResourceDescp + resourceName
    finalListForIndividualResource = []

    # table in database is created based on finalDictionary
    finalDictionary = {}


    # columns of config file transformers is queryFile
    queryFile = []

    # config file consist of data in queryFileDictionary
    queryFileDictionary = {}
    excess = []
    # finalDictionary["version"] = "INT"
    # hardcodefhirPath=resourceName+".meta.versionId"
    # queryFile.append({"columnName": "version","fhirPath":hardcodefhirPath,"columnType": "INT"})

    data = jsondata
    Valstr = ""
    Valtype = ""
    ConcatStr = ""
    fhirPathValstr=""
    ConcatStr1=""

    def func1(data, Valtype, Valstr, ConcatStr,fhirPathValstr,ConcatStr1):
        for key, value in data.items():
            if isinstance(value, dict):
                # print (str(key)+'->'+str(value)+ "*")
                if (str(key) == 'type'):
                    Valtype = 'conct'
                else:
                    Valtype = 'skip'
                func1(value, Valtype, Valstr, ConcatStr,fhirPathValstr,ConcatStr1)
            elif isinstance(value, list):
                if (str(key) == 'fields'):
                    ConcatStr = Valstr
                    ConcatStr1 = fhirPathValstr
                else:
                    ConcatStr = ''
                    ConcatStr1 = ''
                for val in value:
                    typeOfVal = ""
                    try:
                        # print("val====>", val['name'], "val type====>", val["type"]["type"])
                        trial = val["type"]["type"]
                        typeOfVal = 'json'
                        trial = 'BLOB'
                    except:
                        # print("val====>", val['name'], "val type====>", val["type"])
                        # trial = val["type"]
                        if (val["type"] == 'string'):
                            typeOfVal = 'varchar'
                            trial='STRING'
                        elif (val["type"] == 'array'):
                            typeOfVal = 'ARRAY'
                            trial = 'BLOB'
                        elif (val["type"] == 'boolean'):
                            typeOfVal = 'bool'
                            trial = 'BOOLEAN'
                        elif (val["type"] == 'struct'):
                            typeOfVal = 'json'
                            trial = 'BLOB'
                        elif (val["type"] == 'integer'):
                            typeOfVal = 'int'
                            trial = 'INT'
                        else:
                            typeOfVal = 'varchar'
                            trial = 'STRING'
                    if isinstance(val, str):
                        pass
                    elif isinstance(val, list):
                        pass
                    else:

                        if (Valtype == 'conct'):
                            if (ConcatStr == ""):
                                Valstr = val['name']
                                fhirPathValstr = val['name']
                            else:
                                Valstr = ConcatStr + "_" + val['name']
                                fhirPathValstr = ConcatStr1 + "." + val['name']
                        else:
                            if (ConcatStr == ""):
                                Valstr = val['name']
                                fhirPathValstr = val['name']
                            else:
                                Valstr = ConcatStr + "_" + val['name']
                                fhirPathValstr = ConcatStr1 + "." + val['name']

                        newfhirPathValstr = resourceName+"." + fhirPathValstr
                        finalListForIndividualResource.append(Valstr)
                        finalDictionary[str(Valstr)] = typeOfVal

                        # for toinsert dictionary
                        toinsert = {"columnName": "", "fhirPath": "", "columnType": ""}

                        toinsert["columnName"]=Valstr
                        toinsert["fhirPath"]=newfhirPathValstr
                        # toinsert["columnType"]=typeOfVal
                        toinsert["columnType"] = trial
                        # print("toinsert==>",toinsert)
                        if(typeOfVal == 'json'):
                            toinsert["columnName"] = Valstr
                            toinsert["fhirPath"] = newfhirPathValstr + ".first()"
                            toinsert["columnType"] = "STRING"
                            # print("toinsert==>", toinsert)
                            queryFile.append(toinsert)

                        if (trial != 'BLOB'):
                            queryFile.append(toinsert)
                        # print(Valstr)
                        func1(val, Valtype, Valstr, ConcatStr,fhirPathValstr,ConcatStr1)

    func1(data, Valtype, Valstr, ConcatStr,fhirPathValstr,ConcatStr1)
    print("resource====>",resourceName)
    
    finalDictionary['source_resource_id']='varchar'
    finalDictionary['version']='int'
    
    print("finalDictionary===>",finalDictionary)
    # print("queryFile==>",queryFile)

    if(resourceName=="Observation"):
        for pervalue in queryFile:
            if pervalue["columnName"]=="valueBoolean":
                pervalue["fhirPath"]="Observation.value"


    query1 = f"CREATE TABLE {resourceName}()"

    try:
        cur.execute(query1)
    except Exception as e:
        print(resourceName," already exists")
    for j in finalDictionary:
        check=len(j)
        if(check<=63):
            try:
                query2 = f"ALTER TABLE {resourceName} ADD COLUMN {j} {finalDictionary[j]}"
                cur.execute(query2)
            except Exception as e:
                # print("err==>",e)
                pass
        else:
            # excess.append(j)
            jnew = j[0:63]
            try:
                query2 = f"ALTER TABLE {resourceName} ADD COLUMN {jnew} {finalDictionary[j]}"
                cur.execute(query2)
            except Exception as e:
                # print("err==>",e)
                pass
            for index in range(len(queryFile)-1):
                if queryFile[index]['columnName']==j:
                    # del queryFile[index]
                    jnew = j[0:63]
                    queryFile[index]['columnName'] = jnew
                    # print(len(inew), inew)
                    excess.append(jnew)

    conn.commit()

    print("len queryFile==>", len(queryFile))
    queryFileDictionary["columns"]=queryFile
    print("excess",excess)
    # print(queryFileDictionary)

    # Serializing json
    json_object = json.dumps(queryFileDictionary, indent=4)

    # Writing to sample.json
    with open(resourceName, "w") as outfile:
        outfile.write(json_object)




Opened database successfully
length of resource array 1
resource====> Observation
finalDictionary===> {'id': 'varchar', 'id_versioned': 'varchar', 'meta': 'json', 'meta_id': 'varchar', 'meta_versionId': 'varchar', 'meta_versionId_versioned': 'varchar', 'meta_lastUpdated': 'varchar', 'meta_source': 'varchar', 'meta_profile': 'json', 'meta_security': 'json', 'meta_security_id': 'varchar', 'meta_security_system': 'varchar', 'meta_security_version': 'varchar', 'meta_security_code': 'varchar', 'meta_security_display': 'varchar', 'meta_security_userSelected': 'bool', 'meta_tag': 'json', 'meta_tag_id': 'varchar', 'meta_tag_system': 'varchar', 'meta_tag_version': 'varchar', 'meta_tag_code': 'varchar', 'meta_tag_display': 'varchar', 'meta_tag_userSelected': 'bool', 'implicitRules': 'varchar', 'language': 'varchar', 'text': 'json', 'text_id': 'varchar', 'text_status': 'varchar', 'text_div': 'varchar', 'identifier': 'json', 'identifier_id': 'varchar', 'identifier_use': 'varchar', 'identifier_type

# New Section